## Imports

In [8]:
import torch
import torch.utils.data as Data
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torchvision import transforms as T, models
from PIL import Image
from pathlib import Path
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings("ignore")


In [3]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

## Reading/Processing the Data

In [4]:
IMAGE_PATH = Path('./plant-pathology-2020-fgvc7/images')

def image_path(file_stem):
    return IMAGE_PATH/f'{file_stem}.jpg'

In [5]:
train_df = pd.read_csv('./plant-pathology-2020-fgvc7/train.csv')
test_df = pd.read_csv('./plant-pathology-2020-fgvc7/test.csv')

train_paths = train_df['img_file'] = train_df['image_id'].apply(image_path)
test_paths = test_df['img_file'] = test_df['image_id'].apply(image_path)

train_labels = train_df[['healthy','multiple_diseases','rust','scab']]

In [6]:
train_paths, valid_paths, train_labels, valid_labels = train_test_split(
    train_paths, train_labels, test_size = 0.2, random_state=23, stratify = train_labels)
train_paths.reset_index(drop=True,inplace=True)
train_labels.reset_index(drop=True,inplace=True)
valid_paths.reset_index(drop=True,inplace=True)
valid_labels.reset_index(drop=True,inplace=True)

### Creating a custom dataset object

In [7]:
class LeafDataset(Data.Dataset):
    def __init__(self, img_paths, labels, train=True, test=False):
        self.img_paths = img_paths
        self.train = train
        self.test = test
        
        if not self.test:
            self.labels = labels
        
        self.train_transform = T.Compose([T.RandomRotation(25),
                                          T.RandomHorizontalFlip(),
                                          T.RandomVerticalFlip(),])
        self.test_transform = T.Compose([T.RandomRotation(25),
                                          T.RandomHorizontalFlip(),
                                          T.RandomVerticalFlip(),])
        self.default_transform = T.Compose([T.ToTensor(),
                                            T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),]) # ImageNet Stats
        
    def __len__(self):
        return self.img_paths.shape[0]
    
    
    def __getitem__(self, i):
        image = Image.open(self.img_paths[i])
        label = self.labels.loc[i, :].values
        if self.train:
            image = self.train_transform(image)
        elif self.test:
            image = self.test_transform(image)
        image  = self.default_transform(image)
        
        
        return image, label if not self.test else image

### Define training, validataion and testing functions

In [ ]:
def training(model, data_loader):
    pass

def validation(model, data_loader):
    pass

def testing(model, data_loader):
    pass